# Working With Tensors

In this tutorial, we'll go over what you can do with encrypted tensors. Each supported operation will be written out as a function. Then, all of them will be compiled in a loop and executed with a random input to demonstrate their semantics.

### Imports

In [1]:
import concrete.numpy as cnp
import inspect
import numpy as np

### Inputset Definition

We will generate some random input tensors as calibration data for our encrypted tensor functions.

In [2]:
inputset = [np.random.randint(3, 11, size=(3, 2), dtype=np.uint8) for _ in range(10)]

### Supported Operation Definitions

In [3]:
def reshape(x):
    return x.reshape((2, 3))

In [4]:
def flatten(x):
    return x.flatten()

In [5]:
def index(x):
    return x[2, 0]

In [6]:
def slice_(x):
    return x.flatten()[1:5]

In [7]:
def add_scalar(x):
    return x + 10

In [8]:
def add_tensor(x):
    return x + np.array([[1, 2], [3, 3], [2, 1]], dtype=np.uint8)

In [9]:
def add_tensor_broadcasted(x):
    return x + np.array([1, 10], dtype=np.uint8)

In [10]:
def sub_scalar(x):
    return x + (-1)

In [11]:
def sub_tensor(x):
    return x + (-np.array([[1, 2], [3, 3], [2, 1]], dtype=np.uint8))

In [12]:
def sub_tensor_broadcasted(x):
    return x + (-np.array([3, 0], dtype=np.uint8))

In [13]:
def mul_scalar(x):
    return x * 2

In [14]:
def mul_tensor(x):
    return x * np.array([[1, 2], [3, 3], [2, 1]], dtype=np.uint8)

In [15]:
def mul_tensor_broadcasted(x):
    return x * np.array([2, 3], dtype=np.uint8)

In [16]:
def power(x):
    return x ** 2

In [17]:
def truediv(x):
    return x // 2

In [18]:
def dot(x):
    return x.flatten() @ np.array([1, 1, 1, 2, 1, 1], dtype=np.uint8)

In [19]:
def matmul(x):
    return x @ np.array([[1, 2, 3], [3, 2, 1]], dtype=np.uint8)

In [20]:
def clip(x):
    return x.clip(6, 11)

In [21]:
def comparison(x):
    return x > np.array([[10, 5], [8, 11], [3, 7]], dtype=np.uint8)

In [22]:
def minimum(x):
    return np.minimum(x, np.array([[10, 5], [8, 11], [3, 7]], dtype=np.uint8))

In [23]:
def maximum(x):
    return np.maximum(x, np.array([[10, 5], [8, 11], [3, 7]], dtype=np.uint8))

Other than these, we support a lot of numpy functions which you can find more about at [Numpy Support](../howto/numpy_support.md).

### Prepare Supported Operations List 

We will create a list of supported operations to showcase them in a loop.

In [24]:
supported_operations = [
    reshape,
    flatten,
    index,
    slice_,
    add_scalar,
    add_tensor,
    add_tensor_broadcasted,
    sub_scalar,
    sub_tensor,
    sub_tensor_broadcasted,
    mul_scalar,
    mul_tensor,
    mul_tensor_broadcasted,
    power,
    truediv,
    dot,
    matmul,
    clip,
    comparison,
    maximum,
    minimum,
]

### Compilation and Homomorphic Evaluation of Supported Operations

Note that some operations require programmable bootstrapping to work and programmable bootstrapping has a certain probability of failure. Usually, it has more than a 99% probability of success but with big bit-widths, this probability can drop to 95%.

In [25]:
for operation in supported_operations:
    compiler = cnp.Compiler(operation, {"x": "encrypted"})
    circuit = compiler.compile(inputset)
    
    # We setup an example tensor that will be encrypted and passed on to the current operation
    sample = np.random.randint(3, 11, size=(3, 2), dtype=np.uint8)
    result = circuit.encrypt_run_decrypt(sample)
    
    print("#######################################################################################")
    print()
    print(f"{inspect.getsource(operation)}")
    print(f"{operation.__name__}({sample.tolist()}) homomorphically evaluates to {result if isinstance(result, int) else result.tolist()}")
    print()

    expected = operation(sample)
    if not np.array_equal(result, expected):
        print(f"(It should have been evaluated to {expected if isinstance(expected, int) else expected.tolist()} but it didn't due to an error during PBS)")
        print()

#######################################################################################

def reshape(x):
    return x.reshape((2, 3))

reshape([[3, 6], [5, 6], [9, 10]]) homomorphically evaluates to [[3, 6, 5], [6, 9, 10]]

#######################################################################################

def flatten(x):
    return x.flatten()

flatten([[7, 8], [10, 9], [8, 9]]) homomorphically evaluates to [7, 8, 10, 9, 8, 9]

#######################################################################################

def index(x):
    return x[2, 0]

index([[3, 10], [5, 4], [6, 4]]) homomorphically evaluates to 6

#######################################################################################

def slice_(x):
    return x.flatten()[1:5]

slice_([[5, 7], [5, 6], [9, 5]]) homomorphically evaluates to [7, 5, 6, 9]

#######################################################################################

def add_scalar(x):
    return x + 10

add_scalar([[3, 5], [4, 8], [9, 5]]) h